# [Jump Game II](https://leetcode.com/problems/jump-game-ii/)
- Given
    + Array A `[0:N-1]`
        + Always start at `A[0]`
    + A frog at index `i` can jump maximum `A[i]` steps from i
- Find the minimum jumps so that the frog can reach `A[N-1]`


## Examples

#### Example 1

```
Input: A = [2,3,1,1,4]
Output: 2
Explanation: [2] --> [3] --> [4]
```

#### Example 2

```
Input: nums = [2,3,0,1,4]
Output: 2
Explanation: [2] --> [3] --> [4]
```

## Solutions
#### Solution DP

```C++
class Solution {
public:
    int jump(vector<int>& A) {
        int N = A.size();
        
        // dp[i]: Optimal jumps at idx i
        vector<int> dp(N, INT_MAX);
        dp[0] = 0;

        for(int i=0; i<N; ++i) {
            for(int j=i+1; j<= i+A[i]; ++j) {
                if(j >= N) break;
                dp[j] = min(dp[j], dp[i]+1);
            }
        }
        return dp[N-1];
    }
};
```

#### Solution Greedy

```C++
class Solution {
public:
    int jump(vector<int>& A) {
        int N = A.size();

        int jumps = 0, cur_end = 0, cur_farthest = 0;
        for(int i=0; i<N-1; ++i) {
            cur_farthest = max(cur_farthest, i + A[i]);

            if(i == cur_end) {
                jumps += 1;
                cur_end = cur_farthest;
            }
        }
        return jumps;
    }
};
```


# [Jump Game III](https://leetcode.com/problems/jump-game-iii/)
- Given
    + Array A `[0:N-1]`
        + start at index `start`
    + A frog at index `i` can jump
        + foward to index `i + A[i]`
        + or backward to index `i - A[i]`
- Check if the frog can reach `A[i] = 0`

## Examples

#### Example 1

```
Input: arr = [4,2,3,0,3,1,2], start = 5
Output: true
Explanation: 2 ways to reach 
[5]1 -> [4]3 -> [1]2 -> [3]0 
[5]1 -> [6]2 -> [4]3 -> [1]2 -> [3]0

```

#### Example 2

```
Input: arr = [4,2,3,0,3,1,2], start = 0
Output: true 
Explanation:
[0]4 -> [4]3 -> [1]4 -> [3]0
```

#### Example 3

```
Input: arr = [3,0,2,1,2], start = 2
Output: false
````

## Solution
- **Note**: 
    + the state vals is `not monotonic` increasing or decreasing $\to$ No bottom up solution

```C++
class Solution {
public:
    vector<int> A;
    int N;
    vector<bool> vis;

    bool get(int i) {
        if(A[i] == 0) return true;

        vis[i] = true;
        int next_ = i+A[i];
        if(0<=next_ && next_< N && vis[next_] == false) {
            if(get(next_)) return true;
        }

        next_ = i-A[i];
        if(0<=next_ && next_< N && vis[next_] == false) {
            if(get(next_)) return true;
        }

        return false;
    }
    bool canReach(vector<int>& arr, int start) {
        A = arr;
        N = A.size();
        vis.assign(N, false);

        return get(start);
    }
};
```

# [Frog Jump](https://leetcode.com/problems/frog-jump/)
- Given
    + Array `stones` `[0:N-1]`
        + Always start with `stones[0] = 0`
        + `stones[i]`: position of the stone on x axis (always increasing)
    + A frog start at `stones[0]`
        + Can jump to the next stone if the next stone is = `k-1`,`k`, or `k+1` unit distance from the current stone
        + `k` is the distance of the last previous jump
        + 1st jump from stones[0] = 1 unit
- Check if the frog is possible to reach the final stone: `stone[N-1]`

#### Constraints
- N in `[2,1100]`

## Example
#### Examples 1

```
// input
stones = [0,1,3,5,6,8,12,17]

// Output
True

// Explanation
[0] --1--> [1] --2--> [3] --2--> [5] --3--> [8] --4--> [12] --5--> [17] 

```

#### Examples 2

```
// input
stones = [0,1,2,3,4,8,9,11]

// Output
False

// Explanation
[0] --1--> [1] --2--> [3] --1--> [4] --x-->
[0] --1--> [1] --1--> [2] --2--> [4] --x-->

```

## Solution

```C++
class Solution {
public:
    bool canCross(const vector<int>& stones) {
        int N = stones.size();
        if(N == 0) return false;
        if(N == 1 && stones[0] != 0) return false;
        if(N > 1 && (stones[0] != 0 || stones[1] != 1)) return false;

        // dp[i]: if stones[i] can be reached
        vector<bool> dp(N,false);
        dp[0] = true;

        // next_stones[i] = {}: possible next stones from stones[i]
        unordered_map<int, unordered_set<int>>next_stones;
        next_stones[0] = {1};

        // Try stones[j] -> stones[i]
        for(int i=1; i<N; ++i) {
            for(int j=0; j<i; ++j) {
                if(dp[j] == true && next_stones[j].count(stones[i])) {
                    dp[i] = true;

                    int k = stones[i] - stones[j];
                    next_stones[i].insert(stones[i] + k - 1);
                    next_stones[i].insert(stones[i] + k);
                    next_stones[i].insert(stones[i] + k + 1);
                }
            }
        }
        return dp[N-1];
    }
};
```

- Alternative


```C++
class Solution {
public:
    bool canCross(vector<int>& stones) {
        int N = stones.size();
        if(N == 0) return false;
        if(N == 1 && stones[0] != 0) return false;
        if(N > 1 && (stones[0] != 0 || stones[1] != 1)) return false;

        // dp[i]: if stones[i] can be reached
        vector<bool> dp(N,false);
        dp[0] = true;

        // possible_jumps[i] = {}: possible jumps from stones[i]
        unordered_map<int, unordered_set<int>> possible_jumps;
        possible_jumps[0].insert(1);

        // Try stones[j] -> stones[i]
        for(int i=1; i<N; ++i) {
            for(int j=0; j<i; ++j) {
                int k = stones[i] - stones[j];

                if(dp[j] == true && possible_jumps[j].count(k)) {
                    dp[i] = true;

                    possible_jumps[i].insert(k - 1);
                    possible_jumps[i].insert(k);
                    possible_jumps[i].insert(k + 1);
                }
            }
        }

        return dp[N-1];
    }
};
```



# [Minimum Number of Taps to Open to Water a Garden](https://leetcode.com/problems/minimum-number-of-taps-to-open-to-water-a-garden/)
- Given
    + An axis: ----0----N-->
    + An array `ranges` size `N+1`
        + Represent N+1 taps located at `[0, 1, ..., n]`
        + Each tap, `taps[i]` covers a range `[i - ranges[i], i + ranges[i]]`

- Find the minimum number of taps to cover the area `[0, N]`
    +  If the area `[0, N]` cannot be watered return `-1`

#### Constraints
- $1 \leq n \leq 10^4$
- $0 \leq ranges[i] \leq 100$


#### Example 1
```
Input: N = 5, ranges = [3,4,1,1,0,0]
Output: 1
Explanation: The tap at point 0 can cover the interval [-3,3]
The tap at point 1 can cover the interval [-3,5]
The tap at point 2 can cover the interval [1,3]
The tap at point 3 can cover the interval [2,4]
The tap at point 4 can cover the interval [4,4]
The tap at point 5 can cover the interval [5,5]
Opening Only the second tap will water the whole garden [0,5]
```

#### Example 2

```
Input: n = 7, ranges = [1,2,1,0,2,1,0,1]
Output: 3
```


# Solution
- DP relax

```
// Define state: Cover [0, N]
dp[i] = Optimal solution for [0,i]


// Relax state: i -> j
dp[j] = Opt(dp[i] + 1)
    // But the tip is j+ranges[j]
    dp[j+ranges[j]] = Opt(dp[i] + 1)

// Var Range: j,i
j \in [0, N]
i \in [j-ranges[j], j+ranges[j]]
```

- Code

```C++
class Solution {
public:
    int minTaps(int N, vector<int>& ranges) {
        const int INF = 1e5;
        vector<int> dp(N+1, INF);
        dp[0] = 0;

        for(int j=0; j<=N; ++j) for(int i=max(0, j-ranges[j]); i<=min(N, j+ranges[j]); ++i) {
            dp[min(N, j+ranges[j])] = min(dp[min(N, j+ranges[j])], dp[i] + 1);
        }

        return dp[N] == INF ? -1: dp[N];
    }
};
```